In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import font_manager
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# برای نمایش صحیح فارسی/عربی
import arabic_reshaper
from bidi.algorithm import get_display

# ==============================================================================
# Font Configuration for Persian (Farsi) Text
# ==============================================================================

# Add custom font path
font_path = Path(r'D:\OneDrive\AI-Project\Article56\fonts\ttf\Vazirmatn-Regular.ttf')
if font_path.exists():
    font_manager.fontManager.addfont(str(font_path))
    plt.rcParams['font.family'] = 'Vazirmatn'
else:
    print(f"Warning: Font not found at {font_path}")
    print("Falling back to default font...")
    plt.rcParams['font.family'] = 'Tahoma'

# Configure matplotlib for RTL (Right-to-Left) text
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.autolayout'] = True

# ==============================================================================
# تابع کمکی برای تبدیل متن فارسی به فرمت قابل نمایش
# ==============================================================================

def fix_persian_text(text):
    """
    تبدیل متن فارسی/عربی به فرمت قابل نمایش در matplotlib
    
    Parameters:
    -----------
    text : str
        متن فارسی یا عربی
        
    Returns:
    --------
    str
        متن اصلاح‌شده برای نمایش صحیح از راست به چپ
    """
    if text is None or str(text).strip() == '':
        return ''
    
    try:
        # اصلاح شکل حروف (Reshape)
        reshaped_text = arabic_reshaper.reshape(str(text))
        # تبدیل به فرمت راست به چپ (Bidi)
        bidi_text = get_display(reshaped_text)
        return bidi_text
    except Exception as e:
        print(f"Warning: Could not reshape text '{text}': {e}")
        return str(text)

# ==============================================================================
# تابع کمکی برای تبدیل اعداد انگلیسی به فارسی
# ==============================================================================

def convert_to_persian_number(number):
    """
    تبدیل اعداد انگلیسی به اعداد فارسی
    
    Parameters:
    -----------
    number : int, float, str
        عدد یا رشته شامل اعداد انگلیسی
        
    Returns:
    --------
    str
        رشته شامل اعداد فارسی
    """
    # جدول تبدیل اعداد انگلیسی به فارسی
    english_digits = '0123456789'
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    
    # ایجاد جدول ترجمه
    translation_table = str.maketrans(english_digits, persian_digits)
    
    # تبدیل عدد به رشته و سپس به فارسی
    return str(number).translate(translation_table)

def format_number_with_separator(number, use_persian=True):
    """
    قالب‌بندی اعداد با جداکننده هزارگان و تبدیل اختیاری به فارسی
    
    Parameters:
    -----------
    number : int, float
        عدد برای قالب‌بندی
    use_persian : bool
        آیا اعداد به فارسی تبدیل شوند؟ (پیش‌فرض: True)
        
    Returns:
    --------
    str
        عدد قالب‌بندی‌شده
    """
    # قالب‌بندی با جداکننده هزارگان
    formatted = f'{number:,.0f}' if isinstance(number, (int, float)) else str(number)
    
    # تبدیل به اعداد فارسی در صورت نیاز
    if use_persian:
        return convert_to_persian_number(formatted)
    return formatted

# ==============================================================================
# تنظیم مسیر ذخیره‌سازی نمودارها
# ==============================================================================

# ایجاد فولدر fig در صورت عدم وجود
output_dir = Path.cwd() / 'fig'
output_dir.mkdir(exist_ok=True)
print(f"✓ Output directory: {output_dir}")

# ==============================================================================
# Load Data
# ==============================================================================

base_dir = Path.cwd()
data_file = base_dir / 'data' / 'Q_Sample_Data.xlsx'
if not data_file.exists():
    raise FileNotFoundError(f"Expected data file not found: {data_file}")

df = pd.read_excel(data_file)

# Data cleaning and preparation
df['سال'] = df['سال'].astype(int)
df['اعتبار'] = pd.to_numeric(df['اعتبار'], errors='coerce')

# Remove null values in budget column
df = df[df['اعتبار'].notna()]

print(f"Total records: {len(df):,}")
print(f"Year range: {df['سال'].min()} to {df['سال'].max()}")

# ==============================================================================
# Chart 1: Trend of Number of Projects (1398-1403)
# ==============================================================================

fig, ax = plt.subplots(figsize=(14, 8))

# Calculate yearly project counts
yearly_counts = df.groupby('سال').size().sort_index()

# Plot line chart with enhanced styling
ax.plot(yearly_counts.index, yearly_counts.values, 
        marker='o', linewidth=3, markersize=12, 
        color='#2E86AB', markerfacecolor='#2E86AB',
        markeredgewidth=2, markeredgecolor='white',
        label=fix_persian_text('تعداد طرح‌ها'), zorder=3)

# Add values on data points با اعداد فارسی
for x, y in zip(yearly_counts.index, yearly_counts.values):
    # تبدیل عدد به فارسی با جداکننده هزارگان
    persian_number = format_number_with_separator(y, use_persian=True)
    
    ax.text(x, y + max(yearly_counts.values) * 0.02, 
            persian_number, 
            ha='center', va='bottom', 
            fontsize=12, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.4', facecolor='white', 
                     edgecolor='#2E86AB', alpha=0.8))

# Axis labels and title با استفاده از تابع fix_persian_text
ax.set_xlabel(fix_persian_text('سال'), fontsize=14, fontweight='bold', labelpad=10)
ax.set_ylabel(fix_persian_text('تعداد طرح‌های پژوهشی'), fontsize=14, fontweight='bold', labelpad=10)
ax.set_title(fix_persian_text('روند تعداد طرح‌های پژوهشی ماده ۵۶ (۱۳۹۸-۱۴۰۳)'), 
             fontsize=16, fontweight='bold', pad=25)

# Enhanced grid
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8, color='gray')
ax.set_axisbelow(True)

# X-axis configuration با اعداد فارسی
ax.set_xticks(yearly_counts.index)
persian_years = [convert_to_persian_number(year) for year in yearly_counts.index]
ax.set_xticklabels(persian_years, fontsize=12, fontweight='bold')

# Y-axis formatting با اعداد فارسی
def format_y_axis(x, p):
    """فرمت محور Y با اعداد فارسی"""
    return format_number_with_separator(int(x), use_persian=True)

ax.yaxis.set_major_formatter(plt.FuncFormatter(format_y_axis))

# Add background color
ax.set_facecolor('#F8F9FA')
fig.patch.set_facecolor('white')

# Add border
for spine in ax.spines.values():
    spine.set_edgecolor('#CCCCCC')
    spine.set_linewidth(1.5)

# اضافه کردن legend با متن فارسی اصلاح‌شده
ax.legend(loc='upper left', fontsize=12, framealpha=0.9)

plt.tight_layout()

# ذخیره در فولدر fig
output_path = output_dir / 'chart_3_1.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()

print(f"✓ Chart 1 saved: {output_path}")


✓ Output directory: d:\OneDrive\AI-Project\Article56\fig
Total records: 331
Year range: 1397 to 1403
✓ Chart 1 saved: d:\OneDrive\AI-Project\Article56\fig\chart_3_1.png
